# Q4 Transfer Learning with Pre-trained CNN 
Use a pre-trained convolutional neural network (CNN) as a feature extractor and fine-
tune a classifier on a subset of the CIFAR-10 dataset. Set random seeds to 42. Follow
the configuration below

### Imports and seed for reproducibility

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import numpy as np
import random


seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Loading CIFAR-10 with subsets 

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),  # pixel values [0,1]
])

train_full = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_full = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_ds = Subset(train_full, range(2000))
test_ds = Subset(test_full, range(500))

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32)


### Load MobileNetV2 base

In [ ]:

mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenet.features.requires_grad_(False)  # freeze all convolutional layers


# Replace classifier

mobilenet.classifier = nn.Sequential(
    nn.Linear(mobilenet.last_channel, 128),  # 1280 -> 128
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 10)  # 10 classes
)
mobilenet = mobilenet.to(device)

# Optimizer and loss

optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs = 5
for epoch in range(epochs):
    mobilenet.train()
    running_loss = 0
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = mobilenet(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_dl):.4f}")

# Test evaluation

mobilenet.eval()
correct, total = 0, 0
with torch.no_grad():
    for x, y in test_dl:
        x, y = x.to(device), y.to(device)
        pred = mobilenet(x).argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)

print(f"\n Test Accuracy: {100*correct/total:.2f}%")


Epoch 1/5, Loss: 2.1091
Epoch 2/5, Loss: 1.8517
Epoch 3/5, Loss: 1.7231
Epoch 4/5, Loss: 1.6404
Epoch 5/5, Loss: 1.6360

Q4.1 Test Accuracy: 38.80%
